In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!git clone https://github.com/thtrieu/darkflow.git

Cloning into 'darkflow'...
remote: Enumerating objects: 2706, done.
remote: Total 2706 (delta 0), reused 0 (delta 0), pack-reused 2706
Receiving objects: 100% (2706/2706), 32.98 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (1760/1760), done.


In [0]:
cd darkflow

/content/gdrive/My Drive/darkflow


In [0]:
!python3 setup.py build_ext --inplace

Compiling darkflow/cython_utils/nms.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo2_findboxes.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo_findboxes.pyx because it changed.
[1/3] Cythonizing darkflow/cython_utils/cy_yolo2_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/gdrive/My Drive/darkflow/darkflow/cython_utils/cy_yolo2_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/3] Cythonizing darkflow/cython_utils/cy_yolo_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/gdrive/My Drive/darkflow/darkflow/cython_utils/cy_yolo_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[3/3] Cythonizing dar

In [0]:
pip install .

Processing /content/gdrive/My Drive/darkflow
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4ymc8vs/wheels/56/87/2a/00f669ea3d3d308e4fbe5d3f033bf4b54c55c0b90557a32dbb
Successfully built darkflow
  Found existing installation: darkflow 1.0.0
    Uninstalling darkflow-1.0.0:
      Successfully uninstalled darkflow-1.0.0


In [0]:
import matplotlib.pyplot as plt
import numpy as np

from darkflow.net.build import TFNet
import cv2

In [0]:
options = {"model": "cfg/yolo.cfg", "load": "bin/yolo.weights", "threshold": 0.1, "gpu": 1.0}


In [0]:
tfnet = TFNet(options)


Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.4455997943878174s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep

In [0]:
def boxing(original_img, predictions):
    newImage = np.copy(original_img)

    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        if confidence > 0.3:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
            
    return newImage

In [72]:
#CODE FOR PARSING DATA FROM INPUT FOLDER PATH AND THEN STORING OUTPUT IMAGES IN 
#CORRESPONDING OUTPUT PATH WITH CORRESPONDING PARAMETERS STORAGE IN A CSV FILE. 

import os
import pprint as pp
import csv
input_path="./sample_img"
out_path = 'output_images/'
img_cnt=1

with open('output_parameter.csv', mode='w') as employee_file:
  
  for filename in os.listdir(input_path):
    
      if filename.endswith(".jpg") or filename.endswith(".png"): 
        
          img_file=os.path.join(input_path, filename)
          print('\n' + img_file + '\n')
          original_img = cv2.imread(img_file)
          original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
          results = tfnet.return_predict(original_img)

          employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
          employee_writer.writerow([results])
          
          final_img=boxing(original_img, results)
          out_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)
          
          cv2.imwrite(os.path.join(out_path , 'out'+ str(img_cnt) + '.jpg'), out_img)
          img_cnt+=1
          pp.pprint(results)
              
      else:
          continue


./sample_img/sample_computer.jpg

[{'bottomright': {'x': 407, 'y': 314},
  'confidence': 0.15920235,
  'label': 'laptop',
  'topleft': {'x': 325, 'y': 147}},
 {'bottomright': {'x': 420, 'y': 315},
  'confidence': 0.5383351,
  'label': 'cup',
  'topleft': {'x': 320, 'y': 146}},
 {'bottomright': {'x': 483, 'y': 364},
  'confidence': 0.15617442,
  'label': 'bowl',
  'topleft': {'x': 418, 'y': 317}},
 {'bottomright': {'x': 347, 'y': 278},
  'confidence': 0.8700206,
  'label': 'tvmonitor',
  'topleft': {'x': 160, 'y': 95}},
 {'bottomright': {'x': 425, 'y': 374},
  'confidence': 0.31087458,
  'label': 'mouse',
  'topleft': {'x': 373, 'y': 346}},
 {'bottomright': {'x': 334, 'y': 372},
  'confidence': 0.80592436,
  'label': 'keyboard',
  'topleft': {'x': 121, 'y': 261}},
 {'bottomright': {'x': 129, 'y': 356},
  'confidence': 0.45226255,
  'label': 'refrigerator',
  'topleft': {'x': 0, 'y': 17}}]

./sample_img/sample_dog.jpg

[{'bottomright': {'x': 109, 'y': 95},
  'confidence': 0.11069518,
  